In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Statistician%22&l=Los+Angeles%2C+CA&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

325

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

325

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

24

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

Title  \
0   Data Scientist - Statistician                                            
1   Statistician                                                             
2   Reliability Engineer - Statistician                                      
3   Director, Biostatistics, Early Clinical Development                      
4   Principal Statistician                                                   
5   Biostatistician II                                                       
6   Engineer, Analysis/Systems Engineering (Statistician)                    
7   Game Services Engineer                                                   
8   Junior Statistician                                                      
9   Senior Statistician                                                      
10  Assessment & Evaluation Analyst - Kaiser Permanente School of Medicine   
11  Senior Statistician                                                      
12  Statistician                                                             
13  Senior Data Scientist                                                    
14  Data Scientist                                                           
15  Scientific Writer I                                                      
16  Program Director III                                                     
17  Scorekeeper (Statistician)                                               
18  Director, Biostatistics, Early Clinical Development                      
19  Statistician I                                                           
20  Statistician (Health)                                                    
21  Manager, Methodology Statistician                                        
22  ATHLETICS FACILITIES & EVENTS ASSISTANT                                  
23  Research Associate II                                                    

                   Location           City State         Zip     Country  \
0   Glendale, CA             Glendale       CA    None Found  None Found   
1   Beverly Hills, CA 90212  Beverly Hills  CA    90212       None Found   
2   El Segundo, CA 90245     El Segundo     CA    90245       None Found   
3   Los Angeles, CA          Los Angeles    CA    None Found  None Found   
4   Los Angeles, CA 90079    Los Angeles    CA    90079       None Found   
5   Duarte, CA 91010         Duarte         CA    91010       None Found   
6   Canoga Park, CA          Canoga Park    CA    None Found  None Found   
7   Culver City, CA          Culver City    CA    None Found  None Found   
8   Los Angeles, CA 90079    Los Angeles    CA    90079       None Found   
9   Beverly Hills, CA 90212  Beverly Hills  CA    90212       None Found   
10  Pasadena, CA 91101       Pasadena       CA    91101       None Found   
11  Los Angeles, CA 90079    Los Angeles    CA    90079       None Found   
12  La Mirada, CA 90638      La Mirada      CA    90638       None Found   
13  Pasadena, CA 91107       Pasadena       CA    91107       None Found   
14  Pasadena, CA 91107       Pasadena       CA    91107       None Found   
15  Monrovia, CA 91016       Monrovia       CA    91016       None Found   
16  Monrovia, CA 91016       Monrovia       CA    91016       None Found   
17  Los Angeles, CA          Los Angeles    CA    None Found  None Found   
18  Los Angeles, CA          Los Angeles    CA    None Found  None Found   
19  Los Angeles, CA          Los Angeles    CA    None Found  None Found   
20  Los Angeles, CA          Los Angeles    CA    None Found  None Found   
21  Santa Monica, CA         Santa Monica   CA    None Found  None Found   
22  Los Angeles, CA 90032    Los Angeles    CA    90032       None Found   
23  Monrovia, CA 91016       Monrovia       CA    91016       None Found   

                                                                                                                                                                                                                         

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_Statistician_LA.csv')